# Water Turbine Model

The `WaterTurbine` class models a hydropower turbine. It calculates the electrical power generated based on the flow rate of water passing through it and the hydraulic head difference across it. This model is non-stateful in terms of water volume, meaning inflow equals outflow up to the turbine's maximum capacity. This notebook explains the model and shows how to simulate it.

## State Variables

The state of the turbine is described by:

- `outflow` (float): The flow of water passing through the turbine (m^3/s).
- `power` (float): The electrical power generated by the turbine (Watts).

## Parameters

The turbine's performance characteristics are defined by:

- `efficiency` (float): The turbine's conversion efficiency (a value between 0.0 and 1.0).
- `max_flow_rate` (float): The maximum flow rate the turbine can handle (m^3/s).

## Simulation Example

The following example simulates a water turbine operating under a varying inflow and a constant head difference. The simulation demonstrates how the generated power changes with the flow rate. The results, including outflow and power, are plotted.

**Note:** To run this notebook, ensure you have `matplotlib` and `numpy` installed (`pip install matplotlib numpy`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.simulation_identification.physical_objects.water_turbine import WaterTurbine
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the turbine
initial_state = State(outflow=0.0, power=0.0)
parameters = Parameters(
    efficiency=0.85,
    max_flow_rate=50.0 # m^3/s
)

# Create a WaterTurbine instance
turbine = WaterTurbine(name="hydro_turbine_1", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 600  # time step in seconds (10 minutes)
simulation_duration = 3600 * 24  # 1 day
num_steps = int(simulation_duration / dt)

# Store results for plotting
history = []

# Define external conditions
upstream_head = 100.0 # m
downstream_head = 20.0 # m

# Generate a variable inflow pattern (e.g., daily demand cycle)
time_points = np.arange(num_steps)
inflow_pattern = 25 + 25 * np.sin(2 * np.pi * time_points / (24 * 60 / 10)) # Peaking at 50 m^3/s

# Run the simulation loop
for i in range(num_steps):
    inflow = inflow_pattern[i]
    turbine.set_inflow(inflow)
    
    action = {
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = turbine.step(action=action, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", turbine.get_state())

# Prepare data for plotting
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
outflows = [s['outflow'] for s in history]
power_mw = [s['power'] / 1e6 for s in history] # Convert to Megawatts

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_in_hours, outflows, label='Outflow')
ax1.plot(time_in_hours, inflow_pattern, label='Inflow', linestyle='--')
ax1.set_ylabel('Flow Rate (m^3/s)')
ax1.set_title('Water Turbine Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_hours, power_mw, label='Power Generated', color='green')
ax2.set_xlabel('Time (hours)')
ax2.set_ylabel('Power (MW)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
